In [30]:
# import stan #Pystan 3, not available on windows
import os
import model_data as md 
import pandas as pd
import numpy as np
import pickle
import arviz as az 
import sparklyRGT as rgt
pd.options.mode.chained_assignment = None


In [31]:
%load_ext autoreload
%autoreload 2 #deletable cell 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#these variables are for extracting the data using model_data.py

file_names = ['BH07_raw_free_S29-30.xlsx', 'BH04_raw_acquisition.xlsx'] 
df = rgt.load_data(file_names)

fnames = ['']#change these to the names of the csv files

#creates lists of subject #s for each unique task
task_list = df.groupby(['MSN'])['Subject'].unique()


#these lines of code concatenate together the lists of subjects that run the same task (i.e., puts version A 
#and version B together) - based on unique string for each task name

## creates list of subject #s or trial-by-trial data*** --> concatenates together the subjects that run the same task (ex. cuedRGTA and cuedRGTB)
## based on unique string for each task name (ex. cued)

# uncued_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Classic' in task]])
# standard_subs = ...
#...

# subs = [uncued_subs,standard_subs]

numsessions = 5

In [23]:
task_list
df

,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,Trial,...,Pun_Persev_H5,Pun_HeadEntry,Pun_Dur,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H3,Rew_Persev_H4,Rew_Persev_H5
0,rGT_A-cue,2020-10-09 00:00:00,11:01:00,25,0.0,1,0.0,NaN,29,1.0,...,3,3,30,0,0,0,0,0,0,0
1,rGT_A-cue,2020-10-09 00:00:00,11:01:00,25,0.0,1,0.0,NaN,29,2.1,...,0,0,0,1,5,0,0,0,0,0
2,rGT_A-cue,2020-10-09 00:00:00,11:01:00,25,0.0,1,0.0,NaN,29,2.0,...,3,2,30,0,0,0,0,0,0,0
3,rGT_A-cue,2020-10-09 00:00:00,11:01:00,25,0.0,1,0.0,NaN,29,3.0,...,0,0,0,0,0,0,0,0,0,0
4,rGT_A-cue,2020-10-09 00:00:00,11:01:00,25,0.0,1,0.0,NaN,29,4.0,...,2,2,30,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68571,rGT_A-cue,2004-06-19 00:00:00,13:58:53,32,NaN,5,NaN,NaN,27,61.0,...,0,0,0,0,0,0,0,0,0,0
68572,rGT_A-cue,2004-06-19 00:00:00,13:58:53,32,NaN,5,NaN,NaN,27,62.0,...,0,0,0,0,0,0,0,0,0,0
68573,rGT_A-cue,2004-06-19 00:00:00,13:58:53,32,NaN,5,NaN,NaN,27,63.0,...,0,0,0,0,0,0,0,0,0,0
68574,rGT_A-cue,2004-06-19 00:00:00,13:58:53,32,NaN,5,NaN,NaN,27,64.0,...,0,4,40,0,0,0,0,0,0,0


In [ ]:
#change directory to where the csv files are saved
os.chdir('../phd_data')

#use model_data python script to extract data (a bit slow)
uncued = md.get_model_data(fnames, subs[0], numsessions)
standard = md.get_model_data(fnames, subs[1], numsessions)
reverse = md.get_model_data(fnames, subs[2], numsessions)
outcome = md.get_model_data(fnames, subs[3], numsessions)
random = md.get_model_data(fnames, subs[4], numsessions)
loss = md.get_model_data(fnames, subs[5], numsessions)

##model_data takes fnames (same for all variables), list of subject #s and number of sessions (ex. 5, same for all variables)

In [ ]:
#compile stan code
# have to figure out how to pass basic_model as an object, or ideally we can just give it a file name
# have to figure out what random_seed is all about 
basic_standard = stan.build(basic_model, data=uncued, random_seed=1)

#do the sampling
# have to figure out whether num_samples includes warmup
fit_basic_standard = posterior.sample(num_chains=4, num_samples=1000)

## Model_data.py test boxes

In [17]:
def get_model_data(fnames,numsessions, subjects, reset_sessions = True):
    #fnames is a list of strings with the full file locations for each dataset to be loaded in
    df_all = load_multiple_data(fnames, reset_sessions = reset_sessions)
    df = extract_data(df_all, numsessions, subjects)
    df = get_options(df)
    startSubject = start_subject(df)
    startSession = start_session(df)
    model_data = get_data_dict(df, startSubject, startSession)
    return model_data

In [22]:
data = md.load_multiple_data(file_names, reset_sessions = True)
data
#changed subject # (added 100, 200, etc.) and changed session (to start from 1)

,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,Trial,...,Pun_Persev_H5,Pun_HeadEntry,Pun_Dur,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H3,Rew_Persev_H4,Rew_Persev_H5
0,rGT_A-cue,2020-10-09 00:00:00,11:01:00,125,0.0,1,0.0,NaN,1,1.0,...,3,3,30,0,0,0,0,0,0,0
1,rGT_A-cue,2020-10-09 00:00:00,11:01:00,125,0.0,1,0.0,NaN,1,2.1,...,0,0,0,1,5,0,0,0,0,0
2,rGT_A-cue,2020-10-09 00:00:00,11:01:00,125,0.0,1,0.0,NaN,1,2.0,...,3,2,30,0,0,0,0,0,0,0
3,rGT_A-cue,2020-10-09 00:00:00,11:01:00,125,0.0,1,0.0,NaN,1,3.0,...,0,0,0,0,0,0,0,0,0,0
4,rGT_A-cue,2020-10-09 00:00:00,11:01:00,125,0.0,1,0.0,NaN,1,4.0,...,2,2,30,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68571,rGT_A-cue,2004-06-19 00:00:00,13:58:53,232,NaN,5,NaN,NaN,20,61.0,...,0,0,0,0,0,0,0,0,0,0
68572,rGT_A-cue,2004-06-19 00:00:00,13:58:53,232,NaN,5,NaN,NaN,20,62.0,...,0,0,0,0,0,0,0,0,0,0
68573,rGT_A-cue,2004-06-19 00:00:00,13:58:53,232,NaN,5,NaN,NaN,20,63.0,...,0,0,0,0,0,0,0,0,0,0
68574,rGT_A-cue,2004-06-19 00:00:00,13:58:53,232,NaN,5,NaN,NaN,20,64.0,...,0,4,40,0,0,0,0,0,0,0


In [24]:
data.to_excel("load_multiple_data_ex-1.xlsx")

In [ ]:
subjects = [125, 126, 127]
df_extract = md.extract_data_chosen(data, numsessions, subjects)
df_extract
## ran into keyerror for Index(['Option']***
## made a duplicate function that uses chosen (just to test) 

In [39]:
subject_data = md.start_subject(data)
# subject_data

In [43]:
session_data = md.start_session(data)
# session_data

In [49]:
N = len(df.Subject.unique())
ntr = len(df)

C = list(df.Chosen)
R = list(df.Pellets)
P = list(df.Pun_Dur)

model_data = {'N': N, 
                'ntr': ntr,
                'startSubject': subject_data,
                'startSession': session_data,
                'C': C, 
                'R': R, 
                'P': P, }
# model_data
model_data['N']
## model data stores the dict containing N, ntr, etc.
## model_data is all the information we need to pass to the model!!! (C, R, P, and O contain the "data" or "decisions")

32

In [44]:
data_dict = md.get_data_dict(data, subject_data, session_data) 
## creates a dict with all the desired key-value pairs (ex. N, ntr, P, etc.)
## recall: dict is a collection of key-value pairs 

AttributeError: 'DataFrame' object has no attribute 'Option'

## Back to models...

In [ ]:
basic_su = basic.sampling(data = saline_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
basic_rc = basic.sampling(data = rop_cued, iter = 2000, warmup = 1000, chains = 4, control={'max_treedepth': 12})

In [ ]:
basic_ru = basic.sampling(data = rop_uncued, iter = 2000, warmup = 1000, chains = 4, control={'max_treedepth': 12})

In [ ]:
basic2_sc = basic2.sampling(data = saline_cued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
basic2_su = basic2.sampling(data = saline_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
basic2_rc = basic2.sampling(data = rop_cued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
basic2_ru = basic2.sampling(data = rop_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pscale_sc = pscale.sampling(data = saline_cued, iter = 2000, warmup = 1000, chains = 4,
                           control={'adapt_delta': 0.99, 'max_treedepth':12})

In [ ]:
pscale_su = pscale.sampling(data = saline_uncued, iter = 2000, warmup = 1000, chains = 4)

In [ ]:
pscale_rc = pscale.sampling(data = rop_cued, iter = 2000, warmup = 1000, chains = 4, 
                            control={'max_treedepth': 12, 'adapt_delta': 0.99})

In [ ]:
pscale_ru = pscale.sampling(data = rop_uncued, iter = 2000, warmup = 1000, chains = 4, control={'max_treedepth': 12})

In [ ]:
pscale2_sc = pscale2.sampling(data = saline_cued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pscale2_su = pscale2.sampling(data = saline_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pscale2_rc = pscale2.sampling(data = rop_cued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pscale2_ru = pscale2.sampling(data = rop_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pindep_sc = pindep.sampling(data = saline_cued, iter = 2000, warmup = 1000, chains = 4)

In [ ]:
pindep_su = pindep.sampling(data = saline_uncued, iter = 2000, warmup = 1000, chains = 4,
                           control={'adapt_delta': 0.99})

In [ ]:
pindep_rc = pindep.sampling(data = rop_cued, iter = 2000, warmup = 1000, chains = 4,
                           control={'adapt_delta': 0.99})

In [ ]:
print(pindep_rc)

In [ ]:
pindep_ru = pindep.sampling(data = rop_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pindep2_sc = pindep2.sampling(data = saline_cued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pindep2_su = pindep2.sampling(data = saline_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pindep2_rc = pindep2.sampling(data = rop_cued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
pindep2_ru = pindep2.sampling(data = rop_uncued, iter = 1600, warmup = 800, chains = 4)

In [ ]:
save_model('pindep_rc_dict.pkl',pindep,pindep_rc,rop_cued)
save_model('pindep_sc_dict.pkl',pindep,pindep_sc,saline_cued)
save_model('pindep_su_dict.pkl',pindep,pindep_su,saline_uncued)


In [ ]:
save_model('pscale_sc_dict.pkl',pscale,pscale_sc,saline_cued)

In [ ]:
#to save fit:
#model and fit must be saved in dictionary together, with the model first
def save_model(name,model,fit,data):
    model_dict = {'model': model, 'fit': fit, 'data': data}
    with open(name, 'wb') as f:
        pickle.dump(model_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
    


In [ ]:
#load fit:
with open('basic_su_dict.pkl', 'rb') as f:
    basic_su_dict = pickle.load(f)
    
#extract model fit from loaded dictionary
basic_su = basic_su_dict['fit']

In [ ]:
#plotting
az.plot_trace(basic_su)

In [ ]:
#overall summary info
print(basic_su)

In [ ]:
#extract specific parameters 
basic_su_summary = basic_su.extract(permuted = True)
basic_su_etaneg = basic_su_summary['etaNegative']
basic_su_etapos = basic_su_summary['etaPositive']

basic_su_etaneg.mean()

In [ ]:
#convert to pandas dataframe, and save as a csv file

basic_su_df = basic_su.to_dataframe()
basic_su_df.to_csv('LM_saline-uncued_basic-fit')

#load back in
basic_su_df = pd.read_csv('LM_saline-uncued_basic-fit')
